Chapter 3.  Implication
=======

An implication such as $P \Rightarrow Q$ has an antecedent ($P$) and a consequent ($Q$).  If the antecedent is true, the consequent must also be true.  **Prove-It** has an `Implies` **operation** (a sub-class of **Expression**) that may be used to represent an implication, formatted with the $\Rightarrow$ symbol.  While this is a concept used in the core, `Implies` is actually defined outside of the core in the `proveit.logic` package.  It is known in the core for use in the *modus ponens* and *hypothetical reasoning* derivation steps discussed below.

Below we import some helpful and necessary elements, define the `Implies` object $A \Rightarrow B$, and then take a brief tour of the `Implies` class:

In [ ]:
from proveit._common_ import A, B, C, X
from proveit.logic import Implies 
%begin implication

In [ ]:
A_impl_B = Implies(A, B)

Implies is an `Operation` (a sub-class of `Expression`):

In [ ]:
A_impl_B.expr_info()

We can access the `antecedent` and the `consequent` of the implication as follows:

In [ ]:
A_impl_B.antecedent

In [ ]:
A_impl_B.consequent

And we can take a peak at the instance attributes and methods:

In [ ]:
dir(A_impl_B)

## Modus Ponens

*Modus ponens* is a straightforward derivation step in which you can derive $B$ assuming that $A$ and $A \Rightarrow B$ are both true statements.  You can apply this derivation step explicitly by calling the `derive_consequent` method of any `Implies` object.

In [ ]:
B_from_A = A_impl_B.derive_consequent(assumptions=[A, A_impl_B])

Recall from the previous tutorial chapter that **Prove-It**, in its core, is not concerned about truth-aptness.  Neither $A$ nor $B$ are required to be intrinsically truth-apt.  The above **judgment** simply means that $B$ is a true statement *if* we assume that $A$ and $A \Rightarrow B$ are true statements. The associated proof makes explicit the application of modus ponens:

In [ ]:
B_from_A.proof()

In order for the *modus ponens* derivation step to succeed, the implication and the hypothesis must be proven to be true or appear under the applicable assumptions. Below, we see errors in the attempted proof if we omit either of the assumptions $A$ or $A \Rightarrow B$:

In [ ]:
from proveit import ModusPonensFailure
try:
    A_impl_B.derive_consequent(assumptions=[A])
    assert False, "Expecting a ModusPonensFailure error; should not make it to this point"
except ModusPonensFailure as e:
    print("EXPECTED ERROR:", e)

In [ ]:
from proveit import ModusPonensFailure
try:
    A_impl_B.derive_consequent(assumptions=[A_impl_B])
    assert False, "Expecting an ModusPonensFailure error; should not make it to this point"
except ModusPonensFailure as e:
    print("EXPECTED ERROR:", e)

When a **judgment** wraps an **implication**, the **assumptions** of the **judgment** are automatically added.  This is the case for any `Expression` method that accepts an `assumptions` argument and is called indirectly through a `Judgment` object that wraps the `Expression` object.  We demonstrate this in the following two cells:

In [ ]:
A_impl_B_truth = A_impl_B.prove([A_impl_B])

In [ ]:
# A => B is automatically included as an assumption because it is 
# an assumption of A_impl_B_truth
A_impl_B_truth.derive_consequent(assumptions=[A])

### Using an overcomplete set of assumptions

When taking any derivation step under a set of assumptions, any assumptions that are unnecessary will be discarded.  In the example below, we include $C$ as an extra, irrelevant assumption.  It is discarded as it is not needed in the proof.

In [ ]:
A_impl_B.derive_consequent(assumptions=[A_impl_B, A, C]) 

## Hypothetical Reasoning

*Hypothothetical reasoning* is, in some sense, the reverse process of *modus ponens*.  In *modus ponens* the consequent is derived from an implication (and its antecedent).  In *hypothetical reasoning* the implication is derived from the consequent, discarding the antecedent as an assumption.  A common notation to indicate a derivation rule is to display a horizontal line with a new truth below the line that can be derived from what is above the line.  Using this notation, we have

Modus ponens: 
$\begin{array}{c}
\vdash A \Rightarrow B \\
\hline
\{A\} \vdash B
\end{array}$

Hypothetical reasoning: 
$\begin{array}{c}
\{A\} \vdash B \\
\hline
\vdash A \Rightarrow B
\end{array}$

We chose to write these in a form that exhibits the symmetry, though it does not matter whether something is an assumption or a prerequisite truth.  In our *modus ponens* example above, we actually had 

$\begin{array}{c}
\hline
\{A, A \Rightarrow B\} \vdash B
\end{array}$

with no prerequisites, only assumptions.

If $B$ is true assuming $A$, it follows, via *hypothetical reasoning*, that $A \Rightarrow B$.  We prove an implication by assuming the antecedent and deriving the consequent, reasoning through a hypothetical scenario.  This step may be taken by calling `as_implication` (or `as_impl` as an abbreviation) on a `Judgment` object.

We will demonstrate *hypothetical reasoning* by proving the transitivity property of implications, $A \Rightarrow C$ given $A \Rightarrow B$ and $B \Rightarrow C$.   We have already created the `A_impl_B` object to represent $A \Rightarrow B$ and have proven $\{A,A \Rightarrow B\} \boldsymbol{\vdash} B$.  Let us now create a `B_impl_C` object to represent $B \Rightarrow C$:

In [ ]:
B_impl_C = Implies(B, C)

Now we can prove $C$ given $A$, $A \Rightarrow B$, and $B \Rightarrow C$.  This uses the previously derived proof of $\{A,A \Rightarrow B\} \boldsymbol{\vdash} B$ and extends it by deriving the consequent of $B \Rightarrow C$ under appropriate assumptions. 

In [ ]:
CviaTransitivity = B_impl_C.derive_consequent([A, A_impl_B, B_impl_C])

We are now ready to apply *hypothetical reasoning* by calling `as_implication` on this `Judgment**:

In [ ]:
A_impl_C = CviaTransitivity.as_implication(A)

Below is the full **proof** of this **judgment**:

In [ ]:
A_impl_C.proof()

Note that we can take any of the assumptions to be the antecedent and then that assumption will be eliminated:

In [ ]:
A_impl_B__impl__C = CviaTransitivity.as_implication(A_impl_B)

In [ ]:
A_impl_B__impl__C.proof()

In the previous two demonstrations, where the new antecedent was drawn from the set of assumptions, we can think of *hypothetical reasoning* as a procedure in which we transform an "implicit" assumption to an "explicit" antecedent.  Similarly, *modus ponens* may be used to transform an "explicit" antecedent to an "implicit" assumption.  What is the difference between these two different forms of assumption/antecedent?  Why do we need both forms?  The explicit form is necessary because the implicit form cannot be nested.  For example, one could not precisely express $(A \Rightarrow C) \land (B \Rightarrow C) \Rightarrow [(A \lor B) \Rightarrow C]$ with assumptions alone.  The implicit form is also very important.  The implicit form (with assumptions) is extremely convenient, and necessary in the **Prove-It** framework, for accessing the consequent part of an implication directly and applying logical deductions that arise from that consequent (e.g., consider the role of $B$ as a consequent of $A \Rightarrow B$ in the above examples).

The new antecedent does not need to be one of the pre-existing assumptions, however.  After all, a **Judgment** is just as valid when extra assumptions are added (the requirements are simply over-complete).  For example,

In [ ]:
X_impl_C = CviaTransitivity.as_implication(X)

In [ ]:
X_impl_C.proof()

## Automation regarding `Implies` objects

We will talk more generally about automation in the <a href="tutorial08_automation.ipynb">automation</a> chapter, but in this section we will get a preview of that as we look at automation specific to **implications**.

It is not always necessary to call the `derive_consequent` method directly.  In fact, the `derive_consequent` method is called automatically as a "side-effect" whenever a **judgment** for an `Implies` **expression** is created.  In the example below, a **judgment** for $P \Rightarrow Q$ is created via proof-by-assumption which then triggers $Q$ to be derived as a consequent, adding the extra assumption for the antecedent $P$.  Then, a proof for $Q$, under the assumptions of $P \Rightarrow Q$ and $P$, is automatically generated and available upon request:

In [ ]:
from proveit._common_ import P, Q, R, S
QByAssumption = Q.prove([P, Implies(P, Q)])

In [ ]:
QByAssumption.proof()

This can be particularly useful when the proof request is made via some other automation (rather than the manual `prove` request that was demonstrated in the cell above).  This automation is enabled via a `side_effects` method in the `Implies` class which yields `derive_consequent` as a method that should be called when an `Implies` object is created.

In [ ]:
help(Implies.side_effects)

Note that there are several things that may be attempted as automation here, not just `derive_consequent`.  In general, the `side_effects` method of an **Expression** is called, if it exists, whenever a **Judgment** for that **Expression** is created.  The **Judgment** object is passed to this method and the method should yield methods that should be called for deriving desired side-effects.  This enables automation for a variety of **Expression** types, not just `Implies` objects.  Another way that automation may be performed is by implementing a `conclude` method which may attempt to automatically prove a particular type of expression under a given set of assumptions:

In [ ]:
help(Implies.conclude)

The following example of automation relies on the `side_effects` method to populate a dictionary of **Judgment** implications and also on `conclude` to perform a search over these implications to find a path to a conclusion from a hypothesis using intermediate implications via transitivity relations (from $A \Rightarrow B$ and $B \Rightarrow C$, we can obtain $A \Rightarrow C$, as we proved in the previous section). 

In [ ]:
P_impl_S = Implies(P, S).prove([Implies(P, Q), Implies(Q, R), Implies(R, S)])

Below, we display the proof for the above **judgment** that was proven via automation. Notice that the proof relies upon invoking a theorem and applying *instantiation* which will be discussed in detail in later tutorial chapters.  Consider this to be a sneak peak.

In [ ]:
P_impl_S.proof()

We can disable this particular proof and obtain an alternate proof via automation as well.

In [ ]:
P_impl_S.proof().disable()

Note, below, that we do not need to supply the assumptions because `P_impl_S` is a `Judgment` object that automatically includes its own assumptions.

In [ ]:
P_impl_S.prove().proof()

This is a slightly longer proof but it demonstrates that *hypothetical reasoning* can also be automated.  That is, the `Implies.conclude()` method will attempt to apply `as_implication` automatically if other strategies fail (like the previous transitivity approach, which fails because that proof was disabled).

## Disabling/Enabling Automation

If desired, for whatever reason, automation (via `side_effects` and `conclude`) may be disabled by setting the `automation` flag of the `defaults` object to `False`:

In [ ]:
from proveit import defaults
defaults.automation = False

We now attempt an automated proof through transitivity relations similar to what we performed before. In this case we use the reverse direction, $\{S \Rightarrow R, R \Rightarrow Q, Q \Rightarrow P\} \vdash S \Rightarrow P$, to make it different, otherwise it would remember the solution from before.  The proof will fail because automation is disabled.

In [ ]:
from proveit import ProofFailure
try:
    Implies(S, P).prove([Implies(S, R), Implies(R, Q), Implies(Q, P)])
    assert False, "Expecting an ProofFailure error; should not make it to this point"
except ProofFailure as e:
    print("Expected error:", e)

It may be re-enabled by setting this flag back to `True`:

In [ ]:
defaults.automation = True

Now the proof will go through via automation:

In [ ]:
Implies(S, P).prove([Implies(S, R), Implies(R, Q), Implies(Q, P)])

In addition to changing `defaults.automation`, it is also possible to disable automation for a particular instance when calling `prove` (by supplying the argument `automation=False` as part of the method call).  Basically, this just checks if something has been proven already (or is proven by the assumptions or their automatic side-effects) and raises a `ProofFailure` otherwise.  This can be useful in other automation to quickly check a possible proof pathway without potentially wasting the effort to commit to that pathway.

For example, we can check if there already exists a proof for (or automatic side effect giving) $X \Rightarrow Z$ given the assumptions that $X \Rightarrow Y$ and $Y \Rightarrow Z$:

In [ ]:
from proveit._common_ import X, Y, Z
try:
    Implies(X, Z).prove([Implies(X, Y), Implies(Y, Z)], automation=False)
    assert False, "Expecting an ProofFailure error; should not make it to this point"
except ProofFailure as e:
    print("Expected error:", e)

But using the default `automation=True` (*i.e.*, using the "default" by not even supplying the automation argument), we can automatically prove this implication via transitivity (just as we saw above using different labels).

In [ ]:
X_impl_Z = Implies(X, Z).prove([Implies(X, Y), Implies(Y, Z)])

In [ ]:
X_impl_Z.proof()

In [ ]:
%end implication

# Next chapter: <a href="tutorial04_relabeling.ipynb">Relabeling</a>

## <a href="tutorial00_introduction.ipynb#contents">Table of Contents</a>